In [1]:
import pyaerocom as pya
import os
pya.change_verbosity('critical')

/home/jonasg/anaconda3/lib/python3.6/site-packages/geonum/__init__.py:32: UserWarning: Neither LatLon23 nor LatLon are available. Many basic features will not be available (e.g. objects GeoPoint or GeoVector 
  warn('Neither LatLon23 nor LatLon are available. Many basic features '
/home/jonasg/anaconda3/lib/python3.6/site-packages/geonum/__init__.py:42: UserWarning: Plotting of maps etc. is deactivated, please install Basemap
  warn('Plotting of maps etc. is deactivated, please install Basemap')
/home/jonasg/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: The mpl_toolkits.axes_grid module was deprecated in version 2.1. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist provies the same functionality instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


## Comparison of multiple variables based on model suite and observations

This notebook is based on netCDF files that contain data that has been colocated. The colocated data files can be created using the [ANALYSIS_SCRIPT.py](https://github.com/metno/pyaerocom-synthesis/blob/master/ANALYSIS_SCRIPT.py) file which is part of this repository. 

An overview of the available parameters, models and the corresponding obsnetworks used for the intercomparison study is provided below. 

### Setup paths and notebook options

#### Directory for colocated data files

In [2]:
DATA_DIR = pya.const.OUTPUTDIR + '/colocated_data'
# check if read is ok
if not os.access(DATA_DIR, os.R_OK):
    raise IOError('Cannot access data')

#### Comment on colocated data files

The directory containing colocated data files (*DATA_DIR*) contains sub-directories for each model available. The colocated data files are named using the following convention:

*<var_name>_REF-<obsnetwork_id>_MOD-<model_id>_<ts_type_source>_<start_date>_<stop_date>_<ts_type>_<filter_name>_COLL.nc*

For instance:

*abs550aer_REF-AeronetInvV3Lev2.daily_MOD-INCA-BCext_CTRL2016-PD-3hourly_20100101_20101231_monthly_WORLD-noMOUNTAINS_COLL.nc*

#### Output directory

In [3]:
OUTPUT_DIR = 'output_notebooks/'

WRITE_OK = True
if not os.access(OUTPUT_DIR, os.W_OK):
    print('No write access to output directory, no files will be saved')
    WRITE_OK = False

#### Output files

In [4]:
RESULT_TABLE = OUTPUT_DIR + 'multi_comp_table.csv'

#### Options

In [5]:
# If False and RESULT_TABLE exists, then the results are simply read from the table CSV file and not reloaded from the
# colocated data .nc files (faster)
RELOAD_RESULTS = False

# It True, all heatmaps created below are saved as png in the above specified output directory
SAVE_HEATMAPS = True

### Import results 

First, check if results table exists and if not make sure results are reloaded. Then read results using reading class *ReadColocatedData*.

In [6]:
if not os.path.exists(RESULT_TABLE):
    RELOAD_RESULTS = True    

In [ ]:
from helpers.io import ReadColocatedData
reader = ReadColocatedData(DATA_DIR)

if RELOAD_RESULTS:
    table = reader.compute_statistics_table()
    table.to_csv(RESULT_TABLE)
else:
    table = reader.read_statistics_table_csv(RESULT_TABLE)
table #note that this table is also stored in reader.stats_table

/home/jonasg/github/pyaerocom/pyaerocom/mathutils.py:86: RuntimeWarning: divide by zero encountered in true_divide
  tmp = difference / (data + ref_data)
/home/jonasg/github/pyaerocom/pyaerocom/mathutils.py:86: RuntimeWarning: invalid value encountered in true_divide
  tmp = difference / (data + ref_data)


### Analysis of monthly results

In [ ]:
ax=reader.plot_heatmap(colname='NMB', ts_type='monthly', color_rowwise=False,
                       figsize=(20, 12), center=0, vmin=-100, vmax=100, output_dir=OUTPUT_DIR, savefig=SAVE_HEATMAPS)

In [ ]:
reader.plot_heatmap(colname='RMS', ts_type='monthly', cmap='Reds', color_rowwise=True,
                    figsize=(20, 12), output_dir=OUTPUT_DIR, savefig=SAVE_HEATMAPS)

In [ ]:
reader.plot_heatmap(colname='FGE', ts_type='monthly', cmap='Reds', color_rowwise=True,
                    figsize=(20, 12), output_dir=OUTPUT_DIR, savefig=SAVE_HEATMAPS)

### Analysis of yearly results

In [ ]:
ax=reader.plot_heatmap(colname='NMB', ts_type='yearly', color_rowwise=False,
                       figsize=(16, 12), center=0, vmin=-100, vmax=100, output_dir=OUTPUT_DIR, savefig=SAVE_HEATMAPS)

In [ ]:
reader.plot_heatmap(colname='RMS', ts_type='yearly', cmap='Reds', color_rowwise=True,
                    figsize=(16, 12), output_dir=OUTPUT_DIR, savefig=SAVE_HEATMAPS)

In [ ]:
reader.plot_heatmap(colname='FGE', ts_type='yearly', cmap='Reds', color_rowwise=True,
                    figsize=(16, 12), output_dir=OUTPUT_DIR, savefig=SAVE_HEATMAPS)